In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from cooltools.lib.numutils import set_diag
from Bio import SeqIO

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from models import advanced_2d_cnn
from models import simple_1d_cnn

import pandas

pandas.set_option('display.max_columns', 500)
pandas.set_option('display.max_rows', 500)

import h5py
import random

import math
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import cooler
import cooltools as ct

import pickle

import scipy
import os
import re

In [2]:
# should be version 1.x
print(tf.__version__)

1.15.2


In [3]:
# the following directive activates inline plotting
%matplotlib inline

# allow to allocate resources for model training
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

In [4]:
from tensorflow.keras.backend import set_session
sess = tf.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



In [5]:
# UTIL FUNCTIONS

def plot_hic(matrix, use_log_scale = False, chromosome_position = ()):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    
    if use_log_scale:
        im = ax.matshow(np.log10(matrix), cmap='YlOrRd')
        fig.colorbar(im)
    else:
        im = ax.matshow(matrix, cmap='YlOrRd')
        fig.colorbar(im)
    
    if len(chromosome_position) != 0:
        ax.set_title(f"{chromosome_position[0]}: {chromosome_position[1][0]}-{chromosome_position[1][1]}", fontsize=25)

        
def from_upper_triu(vector_repr, matrix_len = 512, num_diags = 2):
    z = np.zeros((matrix_len,matrix_len))
    triu_tup = np.triu_indices(matrix_len,num_diags)
    z[triu_tup] = vector_repr[0, :, 0]
    
    for i in range(-num_diags+1,num_diags):
        set_diag(z, np.nan, i)
        
    return z + z.T


def one_hot_dna(sequence):
    seq_array = np.array(list(sequence))

    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)

    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)

    onehot_encoder = OneHotEncoder(sparse = False)
    result = onehot_encoder.fit_transform(integer_encoded_seq)

    # if Ns are present in the DNA sequence, result will have 5 columns. We delete 4th column which has Ns
    # N row in the resulting training set will have all 0s
    if result.shape[1] == 5:
        result = np.delete(result, 3, 1)

    return result


# combine all the transformations in one function
def transform_hic(hic_matrix, hic_matrix_raw):
    transformed_arr = ct.lib.numutils.adaptive_coarsegrain(hic_matrix, hic_matrix_raw)
    transformed_arr, _, _, _ = ct.lib.numutils.observed_over_expected(transformed_arr, mask = ~np.isnan(transformed_arr))
    transformed_arr = np.log(transformed_arr)
    transformed_arr = ct.lib.numutils.interp_nan(transformed_arr)
    transformed_arr = scipy.ndimage.gaussian_filter(transformed_arr, sigma = 1)
    
    return transformed_arr


def to_upper_triu(input_matrix, diagonal_offset = 2):
    seq_len = input_matrix.shape[0]
    return input_matrix[np.triu_indices(seq_len, diagonal_offset)].reshape([-1, 1])

In [6]:
# Metrics

from sklearn.metrics import mean_squared_error

def rmse(y_orig, y_pred):
    return math.sqrt(mean_squared_error(y_orig, y_pred))


# helper function for the r_squared function
def _squared_error(original, predicted):
    return np.sum((predicted - original) * (predicted - original))


def r_squared(original, predicted):
    y_mean_line = [np.mean(original) for y in original]
    
    squared_error_regr = _squared_error(original, predicted)
    squared_error_y_mean = _squared_error(original, y_mean_line)
    
    return 1 - (squared_error_regr / squared_error_y_mean)

In [7]:
########################
# Simple 1D CNN Model
########################

simple_model = simple_1d_cnn.Model().get_model()

simple_model.load_weights('./weights/project2_model_2.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
########################
# Advanced 2D CNN Model
########################

advanced_model = advanced_2d_cnn.Model().get_model()

advanced_model.load_weights("./weights/model_advanced_weights_1.h5")

In [9]:
##########################################
# Advanced 2D CNN Model with augmentation
##########################################

advanced_model_augmentation = advanced_2d_cnn.Model().get_model()

advanced_model_augmentation.load_weights("./weights/v2/model_80_epochs.h5")

In [10]:
NUM_TEST_SAMPLES = 100

In [17]:
# Test simple model

# On chr2L (which was in training set)

########################################
# Import chr2L DNA sequence and Hi-C map
########################################
seq_chr2L = str(list(SeqIO.parse(open(f"./chromFa/chr2L.fa"),'fasta'))[0].seq).upper()

filepath = "S2-Wang2017-async.dm3.mapq_30.100.mcool"
resolution = "::/resolutions/1000" # 1 KB resolution
c = cooler.Cooler(filepath + resolution)

arr = c.matrix(balance=True).fetch('chr2L')
# For adaptive coarse-grain transformation purposes
arr_raw = c.matrix(balance=False).fetch('chr2L')

transformed_arr_chr2L = transform_hic(arr, arr_raw)

#######

sum_rmse_simple = 0
sum_r2_simple = 0
for i in range(NUM_TEST_SAMPLES):
    print(i)
    offset = i * 50
    
    y_orig = transformed_arr_chr2L[offset:offset+50, offset:offset+50].flatten()
    
    seq_to_predict = one_hot_dna(seq_chr2L[offset * 1000: offset*1000 + 50000]).reshape(1, 50000, 4)
    
    prediction = simple_model.predict(seq_to_predict)
    y_pred = prediction.flatten()
    
    sum_rmse_simple += rmse(y_orig, y_pred)
    sum_r2_simple += r_squared(y_orig, y_pred)
    
    
avg_rmse_simple = sum_rmse_simple / NUM_TEST_SAMPLES
avg_r2_simple = sum_r2_simple / NUM_TEST_SAMPLES


print(avg_rmse_simple)
print(avg_r2_simple)

/opt/anaconda3/lib/python3.7/site-packages/cooltools/lib/numutils.py:1340: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.5422628821408179
-0.3498521799928904


In [15]:
# Test simple model

# On chr4

########################################
# Import chr4 DNA sequence and Hi-C map
########################################
seq_chr4 = str(list(SeqIO.parse(open(f"./chromFa/chr4.fa"),'fasta'))[0].seq).upper()

filepath = "S2-Wang2017-async.dm3.mapq_30.100.mcool"
resolution = "::/resolutions/1000" # 1 KB resolution
c = cooler.Cooler(filepath + resolution)

arr = c.matrix(balance=True).fetch('chr4')
# For adaptive coarse-grain transformation purposes
arr_raw = c.matrix(balance=False).fetch('chr4')

transformed_arr_chr4 = transform_hic(arr, arr_raw)

#######

sum_rmse_simple_chr4 = 0
sum_r2_simple_chr4 = 0
for i in range(NUM_TEST_SAMPLES):
    print(i)
    offset = i * 7 # сдвиг на 7 бинов каждую итерацию (чтобы покрыть всю Hi-C карту за 100 итераций)
    
    y_orig = transformed_arr_chr4[offset:offset+50, offset:offset+50].flatten()
    
    seq_to_predict = one_hot_dna(seq_chr4[offset * 1000: offset*1000 + 50000]).reshape(1, 50000, 4)
    
    prediction = simple_model.predict(seq_to_predict)
    y_pred = prediction.flatten()
    
    sum_rmse_simple_chr4 += rmse(y_orig, y_pred)
    sum_r2_simple_chr4 += r_squared(y_orig, y_pred)
    
    
avg_rmse_simple_chr4 = sum_rmse_simple_chr4 / NUM_TEST_SAMPLES
avg_r2_simple_chr4 = sum_r2_simple_chr4 / NUM_TEST_SAMPLES


print(avg_rmse_simple_chr4)
print(avg_r2_simple_chr4)

/opt/anaconda3/lib/python3.7/site-packages/cooltools/lib/numutils.py:1340: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.5744670787993961
-0.3496494822731242


In [11]:
# Test Advanced model without augmentation

# On all chromosomes except chr4
test_x_advanced = np.load("./train_test_data/test_x.npz")['arr_0']
test_y_advanced = np.load("./train_test_data/test_y.npz")['arr_0']

sum_rmse_advanced = 0
sum_r2_advanced = 0
for i in range(NUM_TEST_SAMPLES):
    print(i)
    
    y_orig = test_y_advanced[i].flatten()
    y_pred = advanced_model.predict(test_x_advanced[i].reshape(1, 524288, 4)).reshape(130305, 1).flatten()
    
    sum_rmse_advanced += rmse(y_orig, y_pred)
    sum_r2_advanced += r_squared(y_orig, y_pred)

    
avg_rmse_advanced_not_chr4 = sum_rmse_advanced / NUM_TEST_SAMPLES
avg_r2_advanced_not_chr4 = sum_r2_advanced / NUM_TEST_SAMPLES

print(avg_rmse_advanced_not_chr4)
print(avg_r2_advanced_not_chr4)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.616260250039287
0.05638696330447948


In [13]:
# Test Advanced model without augmentation

# On chr4

########################################
# Import chr4 DNA sequence and Hi-C map
########################################
seq_chr4 = str(list(SeqIO.parse(open(f"./chromFa/chr4.fa"),'fasta'))[0].seq).upper()

filepath = "S2-Wang2017-async.dm3.mapq_30.100.mcool"
resolution = "::/resolutions/1000" # 1 KB resolution
c = cooler.Cooler(filepath + resolution)

arr = c.matrix(balance=True).fetch('chr4')
# For adaptive coarse-grain transformation purposes
arr_raw = c.matrix(balance=False).fetch('chr4')

transformed_arr_chr4 = transform_hic(arr, arr_raw)

#######

sum_rmse_advanced_chr4 = 0
sum_r2_advanced_chr4 = 0
for i in range(NUM_TEST_SAMPLES):
    print(i)
    offset = i * 7 # сдвиг на 7 бинов каждую итерацию (чтобы покрыть всю Hi-C карту за 100 итераций)
    
    y_orig = to_upper_triu(transformed_arr_chr4[offset:offset+526, offset:offset+526][7:526 - 7, 7:526 - 7]).flatten()
    
    seq_to_predict = one_hot_dna(seq_chr4[offset * 1000: offset*1000 + 526000])[856:(526000 - 856), :].reshape(1, 524288, 4)
    
    prediction = advanced_model.predict(seq_to_predict)
    y_pred = prediction.flatten()
    
    sum_rmse_advanced_chr4 += rmse(y_orig, y_pred)
    sum_r2_advanced_chr4 += r_squared(y_orig, y_pred)
    
    
avg_rmse_advanced_chr4 = sum_rmse_advanced_chr4 / NUM_TEST_SAMPLES
avg_r2_advanced_chr4 = sum_r2_advanced_chr4 / NUM_TEST_SAMPLES


print(avg_rmse_advanced_chr4)
print(avg_r2_advanced_chr4)

/opt/anaconda3/lib/python3.7/site-packages/cooltools/lib/numutils.py:1340: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.8351211958753164
-0.35827105309010954


In [12]:
# Test Advanced model with augmentation

# On all chromosomes except chr4
test_x_advanced_augmentation = np.load("./train_test_data/v2/test_x.npz")['arr_0']
test_y_advanced_augmentation = np.load("./train_test_data/v2/test_y.npz")['arr_0']


sum_rmse_advanced_augmentation = 0
sum_r2_advanced_augmentation = 0
for i in range(NUM_TEST_SAMPLES):
    print(i)
    
    y_orig = test_y_advanced_augmentation[i].flatten()
    y_pred = advanced_model_augmentation.predict(test_x_advanced_augmentation[i].reshape(1, 524288, 4)).reshape(130305, 1).flatten()
    
    sum_rmse_advanced_augmentation += rmse(y_orig, y_pred)
    sum_r2_advanced_augmentation += r_squared(y_orig, y_pred)

    
avg_rmse_advanced_augmentation_not_chr4 = sum_rmse_advanced_augmentation / NUM_TEST_SAMPLES
avg_r2_advanced_augmentation_not_chr4 = sum_r2_advanced_augmentation / NUM_TEST_SAMPLES

print(avg_rmse_advanced_augmentation_not_chr4)
print(avg_r2_advanced_augmentation_not_chr4)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.49051629999215135
0.38661669451125946


In [14]:
# Test Advanced model with augmentation

# On chr4

########################################
# Import chr4 DNA sequence and Hi-C map
########################################
seq_chr4 = str(list(SeqIO.parse(open(f"./chromFa/chr4.fa"),'fasta'))[0].seq).upper()

filepath = "S2-Wang2017-async.dm3.mapq_30.100.mcool"
resolution = "::/resolutions/1000" # 1 KB resolution
c = cooler.Cooler(filepath + resolution)

arr = c.matrix(balance=True).fetch('chr4')
# For adaptive coarse-grain transformation purposes
arr_raw = c.matrix(balance=False).fetch('chr4')

transformed_arr_chr4 = transform_hic(arr, arr_raw)

#######

sum_rmse_advanced_augmentation_chr4 = 0
sum_r2_advanced_augmentation_chr4 = 0
for i in range(NUM_TEST_SAMPLES):
    print(i)
    offset = i * 7 # сдвиг на 7 бинов каждую итерацию (чтобы покрыть всю Hi-C карту за 100 итераций)
    
    y_orig = to_upper_triu(transformed_arr_chr4[offset:offset+526, offset:offset+526][7:526 - 7, 7:526 - 7]).flatten()
    
    seq_to_predict = one_hot_dna(seq_chr4[offset * 1000: offset*1000 + 526000])[856:(526000 - 856), :].reshape(1, 524288, 4)
    
    prediction = advanced_model_augmentation.predict(seq_to_predict)
    y_pred = prediction.flatten()
    
    sum_rmse_advanced_augmentation_chr4 += rmse(y_orig, y_pred)
    sum_r2_advanced_augmentation_chr4 += r_squared(y_orig, y_pred)
    
    
avg_rmse_advanced_augmentation_chr4 = sum_rmse_advanced_augmentation_chr4 / NUM_TEST_SAMPLES
avg_r2_advanced_augmentation_chr4 = sum_r2_advanced_augmentation_chr4 / NUM_TEST_SAMPLES


print(avg_rmse_advanced_augmentation_chr4)
print(avg_r2_advanced_augmentation_chr4)

/opt/anaconda3/lib/python3.7/site-packages/cooltools/lib/numutils.py:1340: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0.8371443574914958
-0.3517389872613376
